In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os

/home/ec2-user/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
direc=os.path.dirname(os.path.realpath('__file__'))
os.path.join(direc)
table = pd.read_csv("cat_prediction_train_ex.csv", names=['userId','age','sex','date','dayofweek','amount','geoId','catId'])
table.sort_values(by='date')
print ('TABLE')
print table


TABLE
    userId  age  sex        date  dayofweek  amount  geoId  catId
0        0   26    0  2018-01-01          0     2.5      3      4
1        0   26    0  2018-01-01          0     2.5      3      8
2        0   26    0  2018-01-01          0     2.5      1      2
3        0   26    0  2018-01-02          0     3.5      7      3
4        0   26    0  2018-01-02          0     2.5      8      4
5        0   26    0  2018-01-04          0     4.0      9      5
6        0   26    0  2018-01-05          0     0.5      1      1
7        1   27    0  2018-02-01          0     2.1      1      1
8        1   27    0  2018-02-01          0     1.6      1      2
9        1   27    0  2018-02-01          0     0.3      1      3
10       1   27    0  2018-02-02          0     0.1      1      4
11       1   27    0  2018-02-03          0     1.1      1      5
12       1   27    0  2018-02-03          0     2.3      1      6
13       1   27    0  2018-02-04          0     2.1      2      7
14  

In [3]:
table['date'] = pd.to_datetime(table['date'], format='%Y-%m-%d')
table = table.set_index(['date'])
print table

            userId  age  sex  dayofweek  amount  geoId  catId
date                                                         
2018-01-01       0   26    0          0     2.5      3      4
2018-01-01       0   26    0          0     2.5      3      8
2018-01-01       0   26    0          0     2.5      1      2
2018-01-02       0   26    0          0     3.5      7      3
2018-01-02       0   26    0          0     2.5      8      4
2018-01-04       0   26    0          0     4.0      9      5
2018-01-05       0   26    0          0     0.5      1      1
2018-02-01       1   27    0          0     2.1      1      1
2018-02-01       1   27    0          0     1.6      1      2
2018-02-01       1   27    0          0     0.3      1      3
2018-02-02       1   27    0          0     0.1      1      4
2018-02-03       1   27    0          0     1.1      1      5
2018-02-03       1   27    0          0     2.3      1      6
2018-02-04       1   27    0          0     2.1      2      7
2018-02-

In [4]:
#min-max scaling 
epsilon = 0.00000000001
table['age'] =    ( table['age'] - table['age'].min() ) /( table['age'].max()-table['age'].min()+epsilon)
table['amount'] = ( table['amount'] - table['amount'].min()) / (table['amount'].max()-table['amount'].min() +epsilon )
print table

            userId  age  sex  dayofweek    amount  geoId  catId
date                                                           
2018-01-01       0  0.0    0          0  0.615385      3      4
2018-01-01       0  0.0    0          0  0.615385      3      8
2018-01-01       0  0.0    0          0  0.615385      1      2
2018-01-02       0  0.0    0          0  0.871795      7      3
2018-01-02       0  0.0    0          0  0.615385      8      4
2018-01-04       0  0.0    0          0  1.000000      9      5
2018-01-05       0  0.0    0          0  0.102564      1      1
2018-02-01       1  1.0    0          0  0.512821      1      1
2018-02-01       1  1.0    0          0  0.384615      1      2
2018-02-01       1  1.0    0          0  0.051282      1      3
2018-02-02       1  1.0    0          0  0.000000      1      4
2018-02-03       1  1.0    0          0  0.256410      1      5
2018-02-03       1  1.0    0          0  0.564103      1      6
2018-02-04       1  1.0    0          0 

In [5]:
#Convert batch
from datetime import datetime

def timeTuple():
    tp = [('2018-01-01','2018-01-03','2018-01-04','2018-01-04'),
          
          ('2018-02-01','2018-02-03','2018-02-04','2018-02-04')
          
          ]            
    return tp


userList = [0,1]

def df2seq(df,userList,params):
    
    #dataframe to Sequence
    #[many2one Sequence maker]
    
    maxLen = params['maxLen']
    categoryCount = params['categoryCount']
    geoCount = params['geoCount']
    
    tp = timeTuple()
    
    seq_train=[] # (batch,maxLen,input_size)
    seq_label=[] # (batch,1) 
    seq_len=[] # (batch)
    seq_mask=[]
    
    for user in userList:
        userHistory = df.groupby(['userId']).get_group((user))
        
        for input_s,input_e, output_s,output_e in tp:
            
            #check-ins summarization based on day
            
            example = userHistory.loc[input_s:input_e]
            answer = userHistory.loc[output_s:output_e]
            
            input_s = pd.Timestamp(input_s)
            output_s= pd.Timestamp(output_s)
            if not example.empty and not answer.empty:
                
                train_window = example.groupby('date')
                infer_window = answer.groupby('date')
                
                cont_features = np.zeros((maxLen,2))
                sex_features = np.zeros((maxLen,2))
                week_features= np.zeros((maxLen,7))
                geo_features = np.zeros((maxLen,geoCount))
                category_features = np.zeros((maxLen,categoryCount))
                
                mask = np.zeros((maxLen,1))
                label = np.zeros((maxLen,1))
                
                LastDay=0
                
                for date,info in train_window:
                    day = (date-input_s).days
                    
                    LastDay = max(LastDay,day)
                    
                    age = float(info.agg({'age':'unique'})['age'])
                    amount = float(info.agg({'amount':'sum'})['amount'])
                    
                    sex = int(info.agg({'sex':'unique'})['sex'])
                    dayofweek = int(info.agg({'dayofweek':'unique'})['dayofweek'])
                    
                    geoList = np.array(info.agg({'geoId':'unique'})['geoId'])
                    catList = np.array(info.agg({'catId':'unique'})['catId'])
                    
                    cont_features[day][0] = age
                    cont_features[day][1] = amount
                    
                    sex_features[day][sex]=1
                    week_features[day][dayofweek]=1
                    
                    geo_features[day][geoList]=1
                    geo_features[day][0]=0
                    
                    category_features[day][catList]=1
                    category_features[day][0]=0
                
                for date,info in infer_window:
                    amount = float(info.agg({'amount':'sum'})['amount'])
                    label[LastDay]+=amount
                    
                
                cont_features = np.array(cont_features)
                sex_features = np.array(sex_features)
                week_features= np.array(week_features)
                geo_features = np.array(geo_features)
                category_features = np.array(category_features)
                
                sequence = np.zeros((maxLen,0))
                sequence = np.concatenate((sequence,cont_features),axis=1)
                sequence = np.concatenate((sequence,sex_features),axis=1) 
                sequence = np.concatenate((sequence,week_features),axis=1)
                sequence = np.concatenate((sequence,geo_features), axis=1)
                sequence = np.concatenate((sequence,category_features),axis=1)
                
                mask[LastDay]=1
                
                seq_train.append(sequence)
                seq_label.append(label)
                seq_mask.append(mask)
                seq_len.append(LastDay+1)
                
                
    return np.array(seq_train),np.array(seq_label),np.array(seq_mask),np.array(seq_len)


userList = [0,1]
params={}
params['maxLen'] = 3  #3일치
params['categoryCount']=10
params['geoCount']=10

params['input_size'] = 2+2+7+params['geoCount']+ params['categoryCount']

seq_train,seq_label,seq_mask,seq_len= df2seq(table,userList,params)

print seq_train
print seq_label
print seq_len
print seq_mask

[[[0.         1.84615385 1.         0.         1.         0.
   0.         0.         0.         0.         0.         0.
   1.         0.         1.         0.         0.         0.
   0.         0.         0.         0.         0.         1.
   0.         1.         0.         0.         0.         1.
   0.        ]
  [0.         1.48717949 1.         0.         1.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   1.         1.         0.         0.         0.         0.
   1.         1.         0.         0.         0.         0.
   0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.        ]]

 [[1.         0.94871795 1.         0.

In [6]:
max_len = params['maxLen']
input_size = params['input_size']

#define lstm
sequence = tf.placeholder( tf.float32, [None, max_len, input_size ])
labels = tf.placeholder( tf.float32, [None,max_len,1]) 
#weights= tf.ones([None])

dynamic_length= tf.placeholder(tf.int32, [None])
cell = tf.contrib.rnn.BasicLSTMCell(num_units = 1)

outputs,_ = tf.nn.dynamic_rnn(cell, sequence , dtype=tf.float32, sequence_length = dynamic_length)

predictions = outputs*seq_mask

loss = tf.reduce_mean(tf.square(labels-predictions))
train = tf.train.AdamOptimizer(learning_rate= 0.051).minimize(loss)

epochs=10
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for it in range(epochs):
        l,_ = sess.run([loss,train],feed_dict={sequence: seq_train,
                                          labels:seq_label,
                                          dynamic_length:seq_len} )

        print ("Loss[%s/%s]:%s"%(it,epochs,l))


Loss[0/10]:0.16775988
Loss[1/10]:0.08688108
Loss[2/10]:0.04376262
Loss[3/10]:0.027055169
Loss[4/10]:0.020529136
Loss[5/10]:0.017486563
Loss[6/10]:0.01525775
Loss[7/10]:0.012896835
Loss[8/10]:0.010182167
Loss[9/10]:0.0072484515
